In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
engine = create_engine("sqlite:///../data/cms.db", echo=True)
conn = engine.connect()

2021-02-02 12:31:57,699 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-02 12:31:57,703 INFO sqlalchemy.engine.base.Engine ()
2021-02-02 12:31:57,706 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-02 12:31:57,707 INFO sqlalchemy.engine.base.Engine ()


In [3]:
#conn.execute("""DROP TABLE CMS""")
# Create the table - note that if we use automap_base, the table must have primary key!
conn.execute("""
CREATE TABLE IF NOT EXISTS cms (
    cms_id INTEGER PRIMARY KEY AUTOINCREMENT ,
    facility_id TEXT NOT NULL ,
    facility_name TEXT NOT NULL ,
    address TEXT NOT NULL ,
    city TEXT NOT NULL ,
    state TEXT NOT NULL ,
    state_name TEXT NOT NULL,
    zip_code TEXT NOT NULL ,
    county_name TEXT NOT NULL ,
    measure_id TEXT  NOT NULL ,
    measure_name TEXT  NOT NULL ,
    denominator INTEGER  NOT NULL ,
    score REAL  NOT NULL ,
    lower_estimate REAL  NOT NULL ,
    higher_estimate REAL  NOT NULL ,
    start_date TEXT NOT NULL ,
    end_date TEXT NOT NULL ,
    household_median_income INTEGER NOT NULL ,
    family_median_income INTEGER NOT NULL ,
    total_population INTEGER NOT NULL ,
    percent_poverty REAL  NOT NULL ,
    percent_veteran REAL  NOT NULL ,
    percent_married REAL  NOT NULL ,
    percent_bachelor REAL  NOT NULL ,
    percent_white REAL  NOT NULL ,
    percent_black REAL  NOT NULL ,
    percent_american_indian REAL  NOT NULL ,
    percent_asian REAL  NOT NULL ,
    percent_hawaiian REAL  NOT NULL ,
    percent_some_other REAL  NOT NULL ,
    percent_two_or_more REAL  NOT NULL ,
    state_code TEXT  NOT NULL ,
    county_code TEXT  NOT NULL 
)
""")

2021-02-02 12:32:00,939 INFO sqlalchemy.engine.base.Engine DROP TABLE CMS
2021-02-02 12:32:00,940 INFO sqlalchemy.engine.base.Engine ()
2021-02-02 12:32:01,065 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-02 12:32:01,067 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE IF NOT EXISTS cms (
    cms_id INTEGER PRIMARY KEY AUTOINCREMENT ,
    facility_id TEXT NOT NULL ,
    facility_name TEXT NOT NULL ,
    address TEXT NOT NULL ,
    city TEXT NOT NULL ,
    state TEXT NOT NULL ,
    state_name TEXT NOT NULL,
    zip_code TEXT NOT NULL ,
    county_name TEXT NOT NULL ,
    measure_id TEXT  NOT NULL ,
    measure_name TEXT  NOT NULL ,
    denominator INTEGER  NOT NULL ,
    score REAL  NOT NULL ,
    lower_estimate REAL  NOT NULL ,
    higher_estimate REAL  NOT NULL ,
    start_date TEXT NOT NULL ,
    end_date TEXT NOT NULL ,
    household_median_income INTEGER NOT NULL ,
    family_median_income INTEGER NOT NULL ,
    total_population INTEGER NOT NULL ,
    percent_poverty REAL  NOT

In [4]:
# Read in target dataframes
census_df = pd.read_csv("../ETL/ETL-Results/census_data.csv", dtype=str)
cms_df = pd.read_csv("../ETL/ETL-Results/four_mort_measures.csv", dtype=str)

In [5]:
# Remove Some Items due to county issues
cms_df = cms_df[cms_df["State"] != "PR"]
cms_df = cms_df[cms_df["State"] != "GU"]
cms_df = cms_df[cms_df["State"] != "MP"]
cms_df = cms_df[cms_df["State"] != "VI"]

In [6]:
len(cms_df)

13478

In [7]:
# Remove Some Items due to county issues
cms_df = cms_df[cms_df["State"] != "PR"]
cms_df = cms_df[cms_df["State"] != "GU"]
cms_df = cms_df[cms_df["State"] != "MP"]
cms_df = cms_df[cms_df["State"] != "VI"]

cms_df = cms_df[cms_df["County Name"] != "OBRIEN"]
cms_df = cms_df[cms_df["County Name"] != "JEFFRSON DAVIS"]

cms_df = cms_df[(cms_df["County Name"] != "LASALLE")
                & (cms_df["State"] != "LA")]
cms_df = cms_df[(cms_df["County Name"] != "ST. MARYS")
                & (cms_df["State"] != "MD")]
cms_df = cms_df[(cms_df["County Name"] != "MCLEAN")
                & (cms_df["State"] != "MD")]

In [8]:
len(cms_df)

13013

In [9]:
def fix_county_name(string):

    if string == "DEKALB":
        return "DE KALB"
    if string == "DUPAGE":
        return "DU PAGE"
    elif string == "DISTRICT OF COLUMBIA":
        return "THE DISTRICT"
    elif string == "ANCHORAGE MUNICIPALITY":
        return "ANCHORAGE"
    elif string == "PRINCE GEORGE\'S":
        return "PRINCE GEORGES"
    elif string == "MCDOWELL":
        return "MC DOWELL"
    elif string == "MCHENRY":
        return "MC HENRY"
    elif string == "LASALLE":
        return "LA SALLE"
    elif string == "MCKEAN":
        return "MC KEAN"
    elif string == "MCDUFFIE":
        return "MC DUFFIE"
    elif string == "EAST BATON ROUGE":
        return "E. BATON ROUGE"
    elif string == "LAPORTE":
        return "LA PORTE"
    elif string == "DEWITT":
        return "DE WITT"
    elif string == "JUNEAU CITY AND BOROUGH":
        return "JUNEAU"
    elif string == "KENAI PENINSULA BOROUGH":
        return "KENAI PENINSULA"
    elif string == "MATANUSKA-SUSITNA BOROUGH":
        return "MATANUSKA-SUSITNA"
    elif string == "DESOTO":
        return "DE SOTO"
    elif string == "DOÑA ANA":
        return "DONA ANA"
    elif string == "LA PAZ":
        return "LAPAZ"
    elif string == "MCLEAN":
        return "MC LEAN"
    elif string == "MCCRACKEN":
        return "MC CRACKEN"
    elif string == "MCDONOUGH":
        return "MC DONOUGH"
    elif string == "KETCHIKAN GATEWAY BOROUGH":
        return "KETCHIKAN GATEWAY"
    elif string == "SCOTTS BLUFF":
        return "SCOTT BLUFF"
    elif string == "NORTHUMBERLAND":
        return "NORTHUMBERLND"
    elif string == "MCLENNAN":
        return "MC LENNAN"
    elif string == "MCMINN":
        return "MC MINN"
    elif string == "FAIRBANKS NORTH STAR BOROUGH":
        return "FAIRBANKS NORTH STAR"
    elif string == "MCLEOD":
        return "MC LEOD"
    elif string == "YELLOW MEDICINE":
        return "YELLOW MEDCINE"
    elif string == "SITKA CITY AND BOROUGH":
        return "SITKA BOROUGH"
    else:
        return string

In [10]:
# Save formatted county names
census_df['County Name'] = census_df['County Name'].apply(
    lambda x: fix_county_name(x))

In [11]:
census_df.columns

Index(['County Name', 'State', 'State Abbr', 'Household Median Income',
       'Family's Median Income', 'Total Population', 'Percent Poverty',
       'Percent Veteran', 'Percent Married', 'Percent Bachelor',
       'Percent One Race White', 'Percent One Race Black+',
       'Percent One Race American Indian+', 'Percent One Race Asian',
       'Percent One Race Hawaiian+', 'Percent One Race Some Other',
       'Percent Two Race Or More', 'State Code', 'County Code'],
      dtype='object')

In [12]:
cms_df.columns

Index(['Facility ID', 'Facility Name', 'Address', 'City', 'State', 'ZIP Code',
       'County Name', 'Measure ID', 'Measure Name', 'Denominator', 'Score',
       'Lower Estimate', 'Higher Estimate', 'Start Date', 'End Date'],
      dtype='object')

In [13]:
cms_merged_df = pd.merge(
    cms_df, census_df, 
    how="inner", left_on=["State", "County Name"], right_on=["State Abbr", "County Name"]
)

In [14]:
cms_merged_df.columns

Index(['Facility ID', 'Facility Name', 'Address', 'City', 'State_x',
       'ZIP Code', 'County Name', 'Measure ID', 'Measure Name', 'Denominator',
       'Score', 'Lower Estimate', 'Higher Estimate', 'Start Date', 'End Date',
       'State_y', 'State Abbr', 'Household Median Income',
       'Family's Median Income', 'Total Population', 'Percent Poverty',
       'Percent Veteran', 'Percent Married', 'Percent Bachelor',
       'Percent One Race White', 'Percent One Race Black+',
       'Percent One Race American Indian+', 'Percent One Race Asian',
       'Percent One Race Hawaiian+', 'Percent One Race Some Other',
       'Percent Two Race Or More', 'State Code', 'County Code'],
      dtype='object')

In [25]:
len(cms_merged_df)

13013

In [15]:
cms_merged_df = cms_merged_df.rename(
        columns={
            "Facility ID": "facility_id",
            "Facility Name": "facility_name",
            "Address": "address",
            "City": "city",
            "State_x": "state",
            "State_y": "state_name",
            "ZIP Code": "zip_code",
            "County Name": "county_name",
            "Measure ID": "measure_id",
            "Measure Name": "measure_name",
            "Denominator": "denominator",
            "Score": "score",
            "Lower Estimate": "lower_estimate",
            "Higher Estimate": "higher_estimate",
            "Start Date": "start_date",
            "End Date": "end_date",
            "Household Median Income": "household_median_income",
            "Family's Median Income": "family_median_income",
            "Total Population": "total_population",
            "Percent Poverty": "percent_poverty",
            "Percent Veteran": "percent_veteran",
            "Percent Married": "percent_married",
            "Percent Bachelor": "percent_bachelor",
            "Percent One Race White": "percent_white",
            "Percent One Race Black+": "percent_black",
            "Percent One Race American Indian+": "percent_american_indian",
            "Percent One Race Asian": "percent_asian",
            "Percent One Race Hawaiian+": "percent_hawaiian",
            "Percent One Race Some Other": "percent_some_other",
            "Percent Two Race Or More": "percent_two_or_more",
            "State Code": "state_code",
            "County Code": "county_code"
        }
)

In [16]:
cms_merged_df.columns

Index(['facility_id', 'facility_name', 'address', 'city', 'state', 'zip_code',
       'county_name', 'measure_id', 'measure_name', 'denominator', 'score',
       'lower_estimate', 'higher_estimate', 'start_date', 'end_date',
       'state_name', 'State Abbr', 'household_median_income',
       'family_median_income', 'total_population', 'percent_poverty',
       'percent_veteran', 'percent_married', 'percent_bachelor',
       'percent_white', 'percent_black', 'percent_american_indian',
       'percent_asian', 'percent_hawaiian', 'percent_some_other',
       'percent_two_or_more', 'state_code', 'county_code'],
      dtype='object')

In [17]:
cms_merged_df = cms_merged_df[[
    "facility_id",
    "facility_name",
    "address",
    "city",
    "state",
    "state_name",
    "zip_code",
    "county_name",
    "measure_id",
    "measure_name",
    "denominator",
    "score",
    "lower_estimate",
    "higher_estimate",
    "start_date",
    "end_date",
    "household_median_income",
    "family_median_income",
    "total_population",
    "percent_poverty",
    "percent_veteran",
    "percent_married",
    "percent_bachelor",
    "percent_white",
    "percent_black",
    "percent_american_indian",
    "percent_asian",
    "percent_hawaiian",
    "percent_some_other",
    "percent_two_or_more",
    "state_code",
    "county_code"        
]]

In [18]:
cms_merged_df.columns

Index(['facility_id', 'facility_name', 'address', 'city', 'state',
       'state_name', 'zip_code', 'county_name', 'measure_id', 'measure_name',
       'denominator', 'score', 'lower_estimate', 'higher_estimate',
       'start_date', 'end_date', 'household_median_income',
       'family_median_income', 'total_population', 'percent_poverty',
       'percent_veteran', 'percent_married', 'percent_bachelor',
       'percent_white', 'percent_black', 'percent_american_indian',
       'percent_asian', 'percent_hawaiian', 'percent_some_other',
       'percent_two_or_more', 'state_code', 'county_code'],
      dtype='object')

In [20]:
cms_merged_df.head(1)

,facility_id,facility_name,address,city,state,state_name,zip_code,county_name,measure_id,measure_name,...,percent_bachelor,percent_white,percent_black,percent_american_indian,percent_asian,percent_hawaiian,percent_some_other,percent_two_or_more,state_code,county_code
0,50704,MISSION COMMUNITY HOSPITAL,14850 ROSCOE BLVD,PANORAMA CITY,CA,California,91402,LOS ANGELES,MORT_30_COPD,Death rate for COPD patients,...,32.5,51.3,8.1,0.7,14.6,0.3,21.0,4.0,06,037


In [21]:
cms_merged_df.to_sql("cms", conn, if_exists="append", index=False)

2021-02-02 12:34:47,691 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("cms")
2021-02-02 12:34:47,692 INFO sqlalchemy.engine.base.Engine ()
2021-02-02 12:34:47,724 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-02 12:34:48,120 INFO sqlalchemy.engine.base.Engine INSERT INTO cms (facility_id, facility_name, address, city, state, state_name, zip_code, county_name, measure_id, measure_name, denominator, score, lower_estimate, higher_estimate, start_date, end_date, household_median_income, family_median_income, total_population, percent_poverty, percent_veteran, percent_married, percent_bachelor, percent_white, percent_black, percent_american_indian, percent_asian, percent_hawaiian, percent_some_other, percent_two_or_more, state_code, county_code) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2021-02-02 12:34:48,122 INFO sqlalchemy.engine.base.Engine (('50704', 'MISSION COMMUNITY HOSPITAL', '14850 ROSCOE BLVD'